In [1]:
import re
import csv
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [2]:
with open('text_classifier', 'rb') as file:
    classifier = pickle.load(file)

In [3]:
with open('leftwingtweets.csv', 'r', newline='') as file:
    X = [tweet for tweet in csv.reader(file)]

In [4]:
from nltk.stem import WordNetLemmatizer
documents = []
stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [5]:
vectorizer = CountVectorizer(max_features=1380, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [6]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [7]:
y_pred = classifier.predict(X)

In [8]:
politicalTweets = []
for i in range(len(y_pred)):
    if y_pred[i] == 1:
        politicalTweets.append([documents[i], 0])

In [9]:
with open('politicalTweets_left.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for tweet in politicalTweets:
        writer.writerow(tweet)